In [25]:
!python --version

Python 3.8.10


In [26]:
sys_prompt = """

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - arguments: Dictionary of arguments for the function_name, which are required for execution of the function. (e.g., device, location, application, title, etc.)

Do not add any superfluous steps. Make sure that each step has all the information needed - do not skip steps.
If it is impossible to generate step by step plan, reply empty list of function. Ensure to enclose the output with "%%%%%%%" after generating with the json format. Do not generate anything after the second %%%%%%%.



"""

In [27]:
sys_prompt_v2 = """

You are a smart home IOT agent.
You will be given a user command enclosed within @@@`Task`@@@ .

You have to generate step by step execution plan for the given user command in json format same as `OUTPUT_FORMAT`.
OUTPUT_FORMAT = !!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!

Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - arguments: Dictionary of arguments for the function_name, which are required for execution of the function. (e.g., device, location, application, title, etc.)

Steps Followed to Generate tool_calls and response:

- Refer `TOOLS` provided to get inference based on description of functions to be used and how to refer their arguments.
- Do not add any superfluous steps. Make sure that each step has all the information needed - do not skip steps.
- If it is impossible to generate step by step plan, reply empty list of function. Ensure to enclose the output with "%%%%%%%" after generating with the json format.
- Generated the tool_calls enclosed with %%%%%%%.
- Generate RESPONSE which is the response from regarding execution of commands, enclosed with %%%%%%%.


[Example: @@@User utterance@@@]

OUTPUT=
<s>
%%%%%%%
{
  "tool_calls": [
    {
      "type": "function",
      "function": [
        {
          "name": `function_name`,
          "arguments": {
              `arguments`
          }
        }
      ]
    }
  ]
}
%%%%%%% TOOLS RESPONSE %%%%%%% I have executed `function_name`
</s>


"""

In [28]:
sys_prompt_implicit_v2 = """

You are a smart home IOT agent.
You will be given a user command enclosed within [`Task`] .

You have to generate step by step execution plan for the given user command in json format same as `OUTPUT_FORMAT`.
OUTPUT_FORMAT = {

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}

Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - arguments: Dictionary of arguments for the function_name, which are required for execution of the function. (e.g., device, location, application, title, etc.)

Steps to generate tool_calls:
  - Refer `TOOLS` provided to get inference based on description of functions to be used and how to refer their arguments.
  - If it is impossible to generate step by step plan, reply empty list of function. Ensure to enclose the output with "%%%%%%%" after generating with the json format.
  - Use homecontext to understand the devices, and only consider devices mentioned in the homecontext data.
  - Before Outputting the deviceId in the functions. Check whether this deviceId is actually there or not in `homeContext`
  - Generated the final output in `OUTPUT_FORMAT` enclosed with %%%%%%%.
  - The `RESPONSE` should be enclosed in %%%%%%%.


[Example: @@@User Utterance@@@]

{
    "homecontext": 
    [
      {
        "deviceId": "tv1_ddd",
        "deviceName": "Television",
        "deviceType": "tv",
        "deviceStatus": "on",
        "location": 
        {
          "locationName": "house",
          "locationId": "l1",
          "roomName": "Living Room",
          "roomId": "r1"
        }
      }
    ]
}

OUTPUT="%%%%%%%
{
  "tool_calls": [
    {
      "type": "function",
      "function": [
        {
          "name": `function_name`,
          "arguments": {
              `arguments`
          }
        }
      ]
    }
  ]
}
%%%%%%% RESPONSE %%%%%%% I can execute this. Should i do it?
"

"""

In [29]:
sys_prompt_v3 = """

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object ONLY contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - arguments: Dictionary of arguments for the function_name, which are required for execution of the function. (e.g., device, location, application, title, etc.)
Only have function name and arguments. Do not output any other fields like description. 

Do not add any superfluous steps. Make sure that each step has all the information needed - do not skip steps.
If it is impossible to generate step by step plan, reply empty list of function.

Use only relavant devices from the homecontext. Do not add devices which are not required.


[Example: @@@User utterance@@@]

OUTPUT=
<s>
%%%%%%%
{
  "tool_calls": [
    {
      "type": "function",
      "function": [
        {
          "name": `function_name`,
          "arguments": {
              `arguments`
          }
        }
      ]
    }
  ]
}
%%%%%%% TOOLS RESPONSE %%%%%%% I have executed `function_name`
</s>


"""

In [30]:
sys_prompt_implicit_v3 = """

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - arguments: Dictionary of arguments for the function_name, which are required for execution of the function. (e.g., device, location, application, title, etc.)

Use homecontext to understand the devices, and only consider devices mentioned in the homecontext data. Do not generate prediction for irrelavant devices.

Do not add any superfluous steps. Make sure that each step has all the information needed - do not skip steps.
If it is impossible to generate step by step plan, reply empty list of function. Ensure to enclose the output with "%%%%%%%" after generating with the json format. Do not generate anything after the second %%%%%%%.


[Example: @@@User utterance@@@]

OUTPUT=
<s>
%%%%%%%
{
  "tool_calls": [
    {
      "type": "function",
      "function": [
        {
          "name": `function_name`,
          "arguments": {
              `arguments`
          }
        }
      ]
    }
  ]
}
%%%%%%% TOOLS RESPONSE %%%%%%% I can execute `function_name`. Should i do it
</s>


"""

In [31]:
# INPUTS

index = 6
max_token_size = 4000
main_prompt = sys_prompt_v3
csvFileName = "results_version_24.csv"

In [32]:
base_model = "custom-finetuned_mistral-7b-instruct-v02-24"
tokenizer_model = "./mistral-7b-instruct-v02"

In [33]:
import torch

print(torch.cuda.is_available())

for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

torch.cuda.empty_cache()

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

True
A100-SXM-80GB


In [34]:
from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')

total    : 85198045184
free     : 66075164672
used     : 19122880512


In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch
from datasets import load_dataset, load_from_disk
from trl import SFTTrainer

In [36]:
device = 'cuda'

In [37]:
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.bfloat16,
        device_map="cuda",
        trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.unk_token
tokenizer.add_eos_token = False
tokenizer.add_bos_token = False
tokenizer.max_length = 8000
tokenizer.truncation = True

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [38]:
model.config.use_cache = True
model.eval()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
       

In [39]:
sys_prompt_implicit = """

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - arguments: Dictionary of arguments for the function_name, which are required for execution of the function. (e.g., device, location, application, title, etc.)

Use homecontext to understand the devices, and only consider devices mentioned in the homecontext data. Do not generate prediction for irrelavant devices.

Do not add any superfluous steps. Make sure that each step has all the information needed - do not skip steps.
If it is impossible to generate step by step plan, reply empty list of function. Ensure to enclose the output with "%%%%%%%" after generating with the json format. Do not generate anything after the second %%%%%%%.



"""

In [40]:
import json
def display(s):
    t = json.loads(s)
    t2 = json.dumps(t,indent=4)
    print(t2)

In [41]:
# sys_prompt = """

# You are a smart home IOT agent.
# You will be given a user command enclosed within @@@`Task`@@@ .

# You have to generate step by step execution plan for the given user command in json format same as `OUTPUT_FORMAT`.
# OUTPUT_FORMAT = !!!
# %%%%%%%
# {

#     "tool_calls": [
#         {
#             "function_name": "function 1",
#             "arguments": 
#                 {
#                     "argument_name 1": "argument value"
#                 }
#          },
#          {
#             "function_name": "function 2",
#             "arguments": 
#                 {
#                     "argument_name 1": "argument value",
#                     "argument_name 2": "argument value"
#                 }
#         }
#     ]
# }
# %%%%%%%
# !!!

# Keywords are defined as below:

# tool_calls: List of tool call objects, where each tool call object contains:  
#     - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
#     - arguments: Dictionary of arguments for the function_name, which are required for execution of the function. (e.g., device, location, application, title, etc.)

# Steps Followed to Generate tool_calls:

# - Refer `TOOLS` provided to get inference based on description of functions to be used and how to refer their arguments.
# - Output the tool_calls and response in the length necessary. If the generated output is repeating stop generating.
# - Do not add any superfluous steps. Make sure that each step has all the information needed - do not skip steps.
# - If it is impossible to generate step by step plan, reply empty list of function. Ensure to enclose the output with "%%%%%%%" after generating with the json format.
# - Generated the tool_calls enclosed with %%%%%%%.


# [Example: @@@Turn on the TV@@@]

# OUTPUT=%%%%%%%
# {
#   "tool_calls": [
#     {
#       "type": "function",
#       "function": [
#         {
#           "name": "identify_device",
#           "arguments": 
#             {
#               "device": "TV"
#             }
#         },
#         {
#           "name": "powerSwitch_getStatus",
#           "arguments":
#             {
#               "device": "#E1"
#             }
#         },
#         {
#           "name": "device_turnOn",
#           "arguments":
#             {
#               "device": "#E1"
#             }
#         }
#       ]
#     }
#   ]
# }
# %%%%%%%

# #### RESPONSE #### I have turned on the TV.

# %%%%%%%

# """

In [42]:
# user_query = """Set the light brightness to 50%"""
# logging.set_verbosity(logging.CRITICAL)

# prompt = sys_prompt + """@@@ """ + user_query + "@@@" + ""
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4000)
# result = pipe(f"<s>[INST] {prompt} [/INST]")
# display(result[0]['generated_text'].split('%%%%%%%')[8])

In [43]:
def model_predict(utt, prompt, index, max_token_size):
    user_query = f"""{utt}"""
    logging.set_verbosity(logging.CRITICAL)

    prompt = prompt + """@@@ """ + user_query + "@@@" + ""
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_token_size)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
#     display(result[0]['generated_text'].split('%%%%%%%')[index])
    return json.loads(result[0]['generated_text'].split('%%%%%%%')[index])

In [44]:
model_predict("Set the light brightness to 50%", main_prompt, index, max_token_size)

{'type': 'function',
 'function': [{'name': 'identify_device', 'arguments': {'device': 'light'}},
  {'name': 'powerSwitch_getStatus', 'arguments': {'device': '#E1'}},
  {'name': 'device_turnOn', 'arguments': {'device': '#E1'}},
  {'name': 'brightness_setValue',
   'arguments': {'device': '#E1', 'brightness': '50'}}]}

In [45]:
user_query = "Set the light brightness to 50%"
logging.set_verbosity(logging.CRITICAL)
prompt = sys_prompt + """@@@ """ + user_query + "@@@"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
# display(result[0]['generated_text'].split('%%%%%%%')[5])

In [46]:
print(result[0]['generated_text'])

<s>[INST] 

You are a smart home IOT agent.
You will be given a user command enclosed within @@@ Task @@@.

You have to generate step by step execution plan for the given user command in json format enclosed between !!!:
!!!
%%%%%%%
{

    "tool_calls": [
        {
            "function_name": "function 1",
            "arguments": 
                {
                    "argument_name 1": "argument value"
                }
         },
         {
            "function_name": "function 2",
            "arguments": 
                {
                    "argument_name 1": "argument value",
                    "argument_name 2": "argument value"
                }
        }
    ]
}
%%%%%%%
!!!
Keywords are defined as below:

tool_calls: List of tool call objects, where each tool call object contains:  
    - function_name: It is name of single execution step for the complete plan. (e.g., identify_device, powerSwitch_getStatus, etc.) Plan can have 1 or more functions in response.
    - argum

In [47]:
import re

def get_utt_slot(data):
    slots = {}
    pattern = re.compile("(\(([^\)]*)\) *\[([^\)]*)\])")
    tag_extracted_nl = pattern.sub(r"\g<2>", data)  # .lower()
    matched_slots = pattern.finditer(data)
    for matched_slot in matched_slots:
        _, raw_slot, tags = matched_slot.groups()
        slots[tags] = raw_slot
    return tag_extracted_nl, slots

In [48]:
def get_intent_slots(result):
    slots = {}
    vi = result['function'][-1]['name'].replace('_', '-')
    for key, value in result['function'][-1]['arguments'].items():
        if isinstance(value, str) and '#E' in value:
            idx = int(value.replace('#E', '')) - 1
            slots[key] = str(result['function'][idx]['arguments'][key])
        else:
            slots[key] = str(value)
    print("vi:", vi, "slots:", slots)
    return vi, slots

In [49]:
res = {
    "type": "function",
    "function": [
        {
            "name": "identify-device",
            "arguments": {
                "device": "light"
            }
        },
        {
            "name": "brightness-increase",
            "arguments": {
                "device": "#E1",
                "brightness": 50
            }
        }
    ]
}

print(get_intent_slots(res))

vi: brightness-increase slots: {'device': 'light', 'brightness': '50'}
('brightness-increase', {'device': 'light', 'brightness': '50'})


In [50]:
def evaluate_result(result, slots, intent):
    pred_vi, pred_slots = get_intent_slots(result)
    vi_result = pred_vi == intent
    if(vi_result == False):
        print(f"[VI] Predicted {pred_vi}, Actual {intent}")
    
    pred_slots, slots = json.dumps(pred_slots, sort_keys=True), json.dumps(slots, sort_keys=True)
    slot_result = pred_slots == slots
    
    if(slot_result == False):
        print(f"[SLOTS] Predicted {pred_slots}, \n Actual{slots}")
    
    print("VI_success:", vi_result, "slot_success:", slot_result)
    
    return vi_result, slot_result, pred_vi, intent, pred_slots, slots

# data = 'Set the (light)[device] brightness to (50)[brightness] %'
# exp_vi = 'brightness-increase'
# utt, slots = get_utt_slot(data)
# model_predict(utt, sys_prompt, 8, 4000)
# print("res", res)
# evaluate_result(res, slots, exp_vi)
    

In [51]:
import os
import json

def read_and_extract_params_from_json_files(directory_path):
    # List to store the extracted parameters
    params_list = []

    # Get a list of all files in the directory
    files = os.listdir(directory_path)

    # Iterate through the list of files
    for file_name in files:
        # Check if the file has a .json extension
        if os.path.splitext(file_name)[1] == '.json':
            # Construct the full file path
            file_path = os.path.join(directory_path, file_name)
            
            # Open the JSON file and load its contents
            with open(file_path, 'r') as file:
                json_data = json.load(file)
                
                # Extract the required parameters
                aligned_nl = json_data.get('utterance', '')
                voice_intent = json_data.get('voiceIntent', '')
                utterance, slots = get_utt_slot(aligned_nl)
                
                # Store the extracted parameters in a dictionary
                params_dict = {'utterance': utterance, 'slots': slots, 'voiceIntent': voice_intent}
                
                # Add the dictionary to the list
                params_list.append(params_dict)
    
    return params_list


# for params in all_params:
#     print(params)

In [52]:
import signal
import time
import json
import csv
from tqdm import tqdm

def timeout_function():
    raise TimeoutError("Function timed out")



#  Evaluate the performance of the model
def evaluate_model_performance(testCaseDir, csvFile):
    directory_path = 'tcs//'
    testCaseParams = read_and_extract_params_from_json_files(testCaseDir)
    
    if not os.path.isfile(os.path.join('ModelPerformanceEvaluation', csvFile)):
        # Create a new empty CSV file
        with open(os.path.join('ModelPerformanceEvaluation', csvFile), 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([])  # Write an empty row to create the CSV structure
        print(f"Created new CSV file: {csvFile}")
    else:
        print(f"CSV file already exists: {csvFile}")
    
    with open(os.path.join('ModelPerformanceEvaluation', csvFile), mode='w', newline='') as csv_file:
        fieldnames = ['utterance', 'Pred_vi', 'Actual_vi', 'Pred_slots', 'Actual_slots', 'vi_result', 'slot_result', 'overall_success']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
              
        overall_success_count = 0
        vi_success_count = 0
        slot_success_count = 0
        total_cases = len(testCaseParams)
        skip = 0
        
        # Create a progress bar object with elapsed time and ETA
        progress_bar = tqdm(total=total_cases, ncols=80, leave=True, mininterval=0.5, unit='it', unit_scale=True)

        # Loop through the range of iterations
        start_time = time.time()  # Record the start time
        for param in testCaseParams:
            model_pred_result = None
            skip_bool = False
            try:
                signal.signal(signal.SIGALRM, timeout_function)
                signal.alarm(10)  # Set alarm for 10 seconds
            #     function to run
                model_pred_result = model_predict(param['utterance'], main_prompt, index, max_token_size)
                signal.alarm(0)  # Cancel the alarm if the function finishes within 10 seconds
            except TimeoutError:
                timout_function()
                skip+=1
                skip_bool = True
            except Exception:
                print("Error")
                skip+=1
                skip_bool = True

            if(model_pred_result == None): continue

    #       Evaluate model performance
#             print(model_pred_result)
            vi_result = False
            slot_result = False
            try:
                vi_result, slot_result, pred_vi, actual_vi, pred_slots, actual_slots = evaluate_result(model_pred_result, param['slots'], param['voiceIntent'])
            except:
                print(f"Utterance could not be classified: {param['utterance']}")

            overall_success = vi_result and slot_result
            overall_success_count += int(overall_success)
            vi_success_count += int(vi_result)
            slot_success_count += int(slot_result)
            
            if(skip_bool):
                vi_result = None
                slot_result = None
                overall_success = None

            writer.writerow({
                'utterance': param['utterance'],
                'Pred_vi': pred_vi,
                'Actual_vi': actual_vi,
                'Pred_slots': pred_slots,
                'Actual_slots': actual_slots,
                'vi_result': vi_result,
                'slot_result': slot_result,
                'overall_success': overall_success
            })
            # Calculate the elapsed time
            elapsed_time = time.time() - start_time

            # Calculate the ETA
            remaining_time = (total_iterations - i) * (elapsed_time / (i + 1))
            eta = time.strftime('%H:%M:%S', time.gmtime(remaining_time))

            # Update the progress bar with elapsed time and ETA
            progress_bar.set_description(f'Elapsed Time: {elapsed_time:.2f}s | ETA: {eta}')
            progress_bar.update(1)



    success_rate = (overall_success_count * 100) / (total_cases - skip)
    vi_success_rate = (vi_success_count * 100) / (total_cases - skip)
    slot_success_rate = (slot_success_count * 100)/ (total_cases - skip)
    print(f"Overall Success Rate: {success_rate:.2f}%")
    print(f"VI Success Rate: {vi_success_rate:.2f}%")
    print(f"Slot Success Rate: {slot_success_rate:.2f}%")
    print(f"Test cases not classified: {skip}")

    # Close the progress bar
    progress_bar.close()

In [53]:
evaluate_model_performance("tcs//", csvFileName)

CSV file already exists: results_version_24.csv


Elapsed Time: 4.01s | ETA: 01:06:46:   0%|    | 1.00/251 [00:04<16:41, 4.01s/it]

vi: device-turnOff slots: {'device': 'cooktop hood'}
VI_success: True slot_success: True


Elapsed Time: 8.01s | ETA: 02:13:25:   1%|    | 2.00/251 [00:08<16:36, 4.00s/it]

vi: device-turnOff slots: {'device': 'cooktop hood'}
VI_success: True slot_success: True


...

vi: deviceMode-setValue slots: {'device': 'air conditioner', 'mode': 'dehumidification'}
[VI] Predicted deviceMode-setValue, Actual airConditioningMode-setValue
[SLOTS] Predicted {"device": "air conditioner", "mode": "dehumidification"}, 
 Actual{"device": "air conditioner", "mode:dry": "dehumidification"}
VI_success: False slot_success: False


Elapsed Time: 1289.21s | ETA: 22:06:46:  98%|█▉| 247/251 [21:29<00:23, 5.91s/it]

vi: washing-startCourse slots: {'device': 'dishes', 'compartment': 'top', 'mode': 'auto'}
[VI] Predicted washing-startCourse, Actual dishWashing-start
[SLOTS] Predicted {"compartment": "top", "device": "dishes", "mode": "auto"}, 
 Actual{"device": "dishwasher"}
VI_success: False slot_success: False


Elapsed Time: 1293.62s | ETA: 23:20:16:  99%|█▉| 248/251 [21:33<00:15, 5.22s/it]

vi: device-turnOn slots: {'device': 'lighting'}
[SLOTS] Predicted {"device": "lighting"}, 
 Actual{"device": "interior lighting", "location": "kitchen"}
VI_success: True slot_success: False
Overall Success Rate: 28.63%
VI Success Rate: 43.15%
Slot Success Rate: 51.61%
Test cases not classified: 3
